# SPCS Networking Connectivity Test: Jira Cloud

**Note: This Notebook should be run in an SPCS Container for testing to be valid**

1. Please provide your Jira URL
2. Check that the Network Rule structure matches your requirements
3. Run the Connectivity Test if your EAI is already configured
4. Use the Cells to prepare the EAI if needed, then rerun the test.

In [ ]:
# Jira Connectivity Test Configuration
# Update the JIRA_INSTANCE_URL with your actual Jira instance URL

# User Configuration - UPDATE THIS
JIRA_INSTANCE_URL = "https://snowflakecomputing.atlassian.net"  # Replace with your actual Jira URL
ATLASSIAN_AUTH_URL = "https://api.atlassian.com"
ATLASSIAN_ID_URL = "https://id.atlassian.com"

# This role will be used to create the EAI and other objects if necessary
IMPLEMENTATION_ROLE = "ACCOUNTADMIN"

# This role will be used by Openflow to run Connectors and perform your Data Engineering tasks
OPENFLOW_RUNTIME_ROLE = "OPENFLOWRUNTIMEROLE"

print(f"Configuration:")
print(f"Jira Instance: {JIRA_INSTANCE_URL}")
print(f"Atlassian Auth: {ATLASSIAN_AUTH_URL}")
print(f"Atlassian ID: {ATLASSIAN_ID_URL}")
print("\nReady to test connectivity...")

In [ ]:
import requests
from urllib.parse import urlparse

def test_connectivity(url, description):
    """Test HTTP connectivity to a given URL"""
    try:
        print(f"\n🔍 Testing {description}: {url}")

        # Simple HTTP test - let EAI handle the routing
        response = requests.get(url, timeout=10, allow_redirects=True)
        print(f"  ✅ HTTP Response: {response.status_code}")

        # Any response means connectivity is working
        return True

    except requests.exceptions.ConnectionError as e:
        print(f"  ❌ Connection Error: {e}")
        return False
    except requests.exceptions.Timeout as e:
        print(f"  ❌ Timeout Error: {e}")
        return False
    except Exception as e:
        print(f"  ❌ Error: {e}")
        return False

# Run connectivity tests
print("=" * 60)
print("JIRA CONNECTIVITY TEST")
print("=" * 60)

results = {}
results['jira_instance'] = test_connectivity(JIRA_INSTANCE_URL, "Jira Instance")
results['atlassian_auth'] = test_connectivity(ATLASSIAN_AUTH_URL, "Atlassian Auth API")
results['atlassian_id'] = test_connectivity(ATLASSIAN_ID_URL, "Atlassian ID Service")

print("\n" + "=" * 60)
print("CONNECTIVITY TEST SUMMARY")
print("=" * 60)

all_passed = True
for service, passed in results.items():
    status = "✅ PASS" if passed else "❌ FAIL"
    print(f"{service.upper()}: {status}")
    if not passed:
        all_passed = False

if all_passed:
    print(f"\n🎉 All connectivity tests PASSED!")
    print("Your SPCS environment can reach all required Jira endpoints.")
    print("You can proceed with Jira integration.")
else:
    print(f"\n⚠️  Some connectivity tests FAILED!")
    print("You need to configure Snowflake External Access Integration (EAI).")
    print("Please run the EAI setup cells below to resolve connectivity issues.")

If Connectivity testing fails, you can use the Cells below to prepare and implement an EAI suitable for JIRA access.

In [ ]:
-- Create Network Rules for Jira Cloud connectivity
-- Run this cell if connectivity tests failed

-- Network rule for Atlassian authentication services
CREATE OR REPLACE NETWORK RULE jira_connectivity_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = (
    '*.atlassian.net:443',
    '*.atlassian.com:443',
    'api.atlassian.com:443',
    'auth.atlassian.com:443',
    'id.atlassian.com:443',
    'api.jira.com:443',
    '{{JIRA_INSTANCE_URL}}:443'
  );

SHOW NETWORK RULES LIKE 'jira_%';

In [ ]:
use role accountadmin;
CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION jira_eai
  ALLOWED_NETWORK_RULES = (
    jira_connectivity_rule
  )
  ENABLED = TRUE
  COMMENT = 'External Access Integration for Atlassian Jira Cloud connectivity';

  SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'jira_eai';

In [ ]:
ALTER NOTEBOOK EAI_JIRA
  SET EXTERNAL_ACCESS_INTEGRATIONS = ('JIRA_EAI');

After Creating and Setting the EAI on the Notebook, run the Connectivity test again.

*NOTE: Remember to restart your compute session*

In [ ]:
-- Check if network rules exist
SHOW NETWORK RULES LIKE 'jira_%';

In [ ]:
-- Check if EAI exists
SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'jira_eai';